In [1]:
%matplotlib widget

import sys
sys.path.insert(0,'../../src')

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from process_results import *
from dataset_tester import *
from get_landmarks import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from scipy import linalg
import seaborn as sns

In [2]:
MEEI_path = Path('/media/2TB/data/bp_pat_datasets/MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

In [3]:
pat_info = pd.read_pickle('../../src/pickles/MEEI_pat_info.pkl')

with open('../../src/pickles/all_pats_processed_data.pickle','rb') as f:
    loaded_pats = pickle.load(f)

EOFError: Ran out of input

In [4]:
pat_info

,Category,Sub-category,#,Side,Gender,Age,Path,HB_scale
0,Flaccid,Complete,1,Left,Male,54,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
1,Flaccid,Complete,2,Right,Female,16,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
2,Flaccid,Complete,3,Right,Male,38,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
3,Flaccid,Complete,4,Left,Female,31,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
4,Flaccid,Complete,5,Left,Female,52,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
5,Flaccid,Mild,1,Left,Male,72,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
6,Flaccid,Mild,2,Right,Female,55,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
7,Flaccid,Mild,3,Left,Male,75,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
8,Flaccid,Moderate,1,Left,Male,51,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4
9,Flaccid,Moderate,3,Left,Female,62,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4


In [5]:
# x - choose one
all_ids = loaded_pats[0]
all_imgs = loaded_pats[1]
all_lms = loaded_pats[2]
all_features = loaded_pats[3]

# y
all_hb = loaded_pats[4]

NameError: name 'loaded_pats' is not defined

In [6]:
# Find NaN and Inf values
bad_indices = np.where(np.isnan(all_features).sum(axis=1) > 0)[0]
bad_indices = np.append(bad_indices, np.where(np.isinf(all_features).sum(axis=1) > 0)[0])
bad_indices.sort()
bad_indices

NameError: name 'all_features' is not defined

In [7]:
# drop bad indices 
# x
all_ids = np.delete(all_ids, bad_indices, axis=0)
all_imgs = np.delete(all_imgs, bad_indices, axis=0)
all_lms = np.delete(all_lms, bad_indices, axis=0)
all_features = np.delete(all_features, bad_indices, axis=0)

# y
all_hb = np.delete(all_hb, bad_indices, axis=0)

all_features.shape, all_hb.shape

NameError: name 'all_ids' is not defined

In [8]:
plot_df = pd.DataFrame(all_features, columns = ['f'+str(i) for i in range(0, 29)])

NameError: name 'all_features' is not defined

In [9]:
plot_df['hb'] = all_hb

NameError: name 'all_hb' is not defined

In [ ]:
def feature_plot(df, which_feature, group_by):
    sns.displot(df, x=which_feature, hue=group_by, element="step")
    

In [ ]:
for i in range(0, 29):
    feature_plot(plot_df, 'f'+str(i), 'hb')

In [ ]:
# Split IDs into train and test
random.seed(85)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 40)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

In [ ]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train, x_test = all_features[train_idx], all_features[test_idx]
y_train, y_test = all_hb[train_idx], all_hb[test_idx]

print('Training x shape:', x_train.shape)
print('Training y shape:', y_train.shape)
print('Testing x shape:', x_test.shape)
print('Testing y shape:', y_test.shape)

In [ ]:
x_train[0], y_train[0]

In [ ]:
x_test[0], y_test[0]

In [ ]:
#data_dmatrix = xgb.DMatrix(data=all_features,label=all_hb)

In [ ]:
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 8, alpha = 10, n_estimators = 30)

In [ ]:
xg_reg = xgb.XGBClassifier(
    tree_method="gpu_hist", enable_categorical=True, n_estimators=30, max_depth=10)

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
xg_reg.fit(x_train, y_train)

In [ ]:
y_pred = xg_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %2f" % (rmse))
print('Accuracy:', round(accuracy_score(y_test, y_pred), 3))

In [ ]:
y_pred[10:20], y_test[10:20]